In [3]:
import pandas as pd
import hail as hl
import os
import re
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from datetime import datetime
from tqdm import tqdm

## Load data - for data loading we have two options:
1. Load file with e-code, disease and one result
2. Load file with e-code, disease and multiple results (mapped with corresponding NM codes)

In [4]:
def parse_phenotype_file(filepath):
    # Read the TSV file with UTF-8 encoding
    df = pd.read_csv(filepath, delimiter='\t', header=None, encoding='ISO-8859-1')
    df.columns = ['E_code', 'Disease', 'Result']  # Setting column names according to the expected format

    # Replace any nan values in 'Result' with 'NA'
    df['Result'] = df['Result'].fillna('NA')

    # Store Disease and Result in a tuple for each E_code
    aggregated_dict = {row['E_code']: (row['Disease'], row['Result']) for index, row in df.iterrows()}
    print("Aggregated data parsed successfully.")
    print(aggregated_dict)
    return aggregated_dict

phenotype_data= parse_phenotype_file('/mnt/sdb/phenotype-globals-tshc-7500.tsv') # file for one result per e-code

Aggregated data parsed successfully.
{'E01978327': ('TERVE', 'NM_000051.4(ATM):c.3993+1G>A, p.?'), 'E02081219': ('RV', 'NM_000051.4(ATM):c.5932G>T, p.(Glu1978*)'), 'E02081256': ('RV', 'NM_000051.4(ATM):c.5932G>T, p.(Glu1978*)'), 'E02179659': ('RV', 'NM_000051.4(ATM):c.7629+1G>A, p.?'), 'E02179721': ('RV', 'NM_000051.4(ATM):c.1369C>T, p.(Arg457*)'), 'E01853442': ('RV MÕLEMAS', 'NM_000051.4(ATM):c.3304G>T, p.(Gly1102*)'), 'E01854344': ('TERVE', 'NM_000051.4(ATM):c.7471T>G, p.(Trp2491Gly)'), 'E01855161': ('RV', 'NM_000051.4(ATM):c.5932G>T, p.(Glu1978*)'), 'E01716033': ('RV', 'NM_000051.4(ATM):c.8147T>C, p.(Val2716Ala)'), 'E01482746': ('TERVE', 'NM_000051.4(ATM):c.2639G>T, p.(Gly880Val)'), 'E01604610': ('RV', 'NM_000051.4(ATM):c.1339C>T, p.(Arg447*)'), 'E01482832': ('RV', 'NM_007294.4(BRCA1):c.4258C>T, p.(Gln1420*)'), 'E01483236': ('RV', 'NM_007294.4(BRCA1):c.5266dup, p.(Gln1756Profs*74)'), 'E01483982': ('TERVE', 'NM_007294.4(BRCA1):c.5117G>A,\xa0p.(Gly1706Glu)'), 'E01484081': ('RV', 'NM_0

In [6]:
## Siit uus elu
import pandas as pd
import re

def extract_nm_identifiers(expression):
    if pd.isna(expression):
        return []
    else:
        # Extract NM and other identifiers using regex
        return re.findall(r'(?:LRG_\d+t\d+|NM_\d+\.\d+)(?::c\.[^\s,]+)?', expression)

def parse_phenotype_file_HGSV(filepath, additional_nucleotide=True):
    # Load data
    data = pd.read_csv(filepath, sep='\t', encoding='utf-8')

    # Create a dictionary to store extracted data
    extracted_data = {}

    # Iterate over rows
    for index, row in data.iterrows():
        e_code = row['E_code']
        disease = row['Disease']
        symbol = row['#Symbol']
        nucleotide_expression = row['NucleotideExpression']
        additional_nucleotide_expressions = row['Additional_NucleotideExpressions']

        # Extract NM identifiers
        nm_identifiers = extract_nm_identifiers(additional_nucleotide_expressions)

        # Ensure unique and ordered identifiers to avoid duplication
        unique_identifiers = sorted(set(nm_identifiers), key=lambda x: x.split(':c.')[1] if ':c.' in x else x)

        # Store the extracted data, changing from list to tuple
        if unique_identifiers and additional_nucleotide:
            extracted_data[e_code] = (disease, symbol, nucleotide_expression, *unique_identifiers)
        else:
            extracted_data[e_code] = (disease, symbol, nucleotide_expression)

    return extracted_data

# Example usage
file_path = '/mnt/sdb/markus-bsc-thesis-data/cleaned-HGSV-phenotype-globals-tshc-7500.tsv'
phenotype_data = parse_phenotype_file_HGSV(file_path, True)

# Print only the first 20 entries from the extracted data
for i, (e_code, info) in enumerate(phenotype_data.items()):
    if i >= 20:
        break
    print(f"{e_code}: {info}")

E01978327: ('TERVE', nan, nan)
E02081219: ('RV', 'ATM', 'LRG_135t1:c.5932G>T', 'NM_001351110.2:c.*39-3353C>A', 'NM_000051.3:c.5932G>T', 'NM_001351834.2:c.5932G>T', 'LRG_135t1:c.5932G>T', 'NM_000051.4:c.5932G>T', 'NM_001330368.2:c.641-3353C>A')
E02081256: ('RV', 'ATM', 'LRG_135t1:c.5932G>T', 'NM_001351110.2:c.*39-3353C>A', 'NM_000051.3:c.5932G>T', 'NM_001351834.2:c.5932G>T', 'LRG_135t1:c.5932G>T', 'NM_000051.4:c.5932G>T', 'NM_001330368.2:c.641-3353C>A')
E02179659: ('RV', 'ATM', 'NM_000051.3:c.7629+1G>A', 'NM_152587.2:c.*1279C>T', 'NM_001351110.2:c.*38+3662C>T', 'NM_001330368.2:c.641-22487C>T', 'NM_000051.4:c.7629+1G>A', 'NM_000051.3:c.7629+1G>A', 'LRG_135t1:c.7629+1G>A', 'NM_001351834.2:c.7629+1G>A')
E02179721: ('RV', 'ATM', 'NM_000051.4:c.1369C>T', 'NM_000051.4:c.1369C>T', 'NM_000051.3:c.1369C>T', 'NM_001351834.2:c.1369C>T', 'LRG_135t1:c.1369C>T')
E01853442: ('RV MÕLEMAS', nan, nan)
E01854344: ('TERVE', nan, nan)
E01855161: ('RV', 'ATM', 'LRG_135t1:c.5932G>T', 'NM_001351110.2:c.*39-335

## Prepare data for classification and converting them to TSV files 

In [ ]:
def classify_disease(disease, position):
    if disease == 'RV' and (position != 'NA' and position != 'NEG'):
        return 'positive-group-RV'
    else:
        return 'negative-group'


def parse_empty(text):
    """Converts an empty string to a missing value, otherwise converts to integer."""
    return hl.if_else((text == "") | hl.is_missing(text) | ~text.matches(r"\d+"), hl.missing(hl.tint32), hl.int32(text))


def safe_float_parse(s):
    """Parse a float safely, return None if parsing fails due to non-numeric string."""
    return hl.if_else(hl.is_defined(s) & s.matches(r'^-?\d*(\.\d+)?$'), hl.float64(s), hl.missing(hl.tfloat64))


def parse_to_int32(text):
    return hl.if_else((text == "") | (text == "null"), hl.missing(hl.tint32), hl.int32(text))


def parse_to_float64(text):
    # This regex will match numbers including integers, floats, and also consider negative values
    return hl.if_else(hl.is_defined(text) & text.matches(r'^-?\d*\.?\d+(?:[Ee][-+]?\d+)?$'), hl.float64(text),
                      hl.missing(hl.tfloat64))


def safe_index(split_list, index, default=None):
    try:
        return split_list[index]
    except IndexError:
        return default


def vcfs_to_matrixtable(source, phenotype_data, base_destination, write=True,
                        log_file='/home/markus/gen-toolbox/output/processed_vcfs_log.tsv'):
    files = []

    if os.path.isdir(source):
        files = [os.path.join(source, f) for f in os.listdir(source) if f.endswith('.vcf') or f.endswith('.vcf.gz')]
    elif os.path.isfile(source) and (source.endswith('.vcf') or source.endswith('.vcf.gz')):
        files.append(source)
    else:
        raise ValueError("Invalid path or file type. Must be a directory or a VCF file.")

    hl.init(default_reference='GRCh37')  # Initialize Hail
    # TODO: peame välja manama selle 
    # Contig recoding for import_vcf
    contig_recoding = {f"chr{i}": str(i) for i in range(1, 23)}
    contig_recoding.update({"chrX": "X", "chrY": "Y"})

    log_entries = []
    processed_vcfs = []

    # Import and annotate files in a loop
    try:
        for vcf in tqdm(files, desc="Processing VCFs"):

            patient_code = os.path.basename(vcf).split('_')[0]
            phenotype_info = phenotype_data.get(patient_code, ("NA", "NA"))

            disease, position = phenotype_info
            group = classify_disease(disease, position)

            destination = os.path.join(base_destination, group)
            destination_path = os.path.join(destination, os.path.basename(vcf).replace('.vcf', '.mt'))
            tsv_output_path = os.path.join(destination, f"{patient_code}_mtoutput.tsv")

            if os.path.exists(destination_path) or os.path.exists(tsv_output_path):
                print(f"Skipping {vcf}, as the output file already exists in {destination_path}.")
                log_entries.append({
                    "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "VCF": vcf,
                    "Status": "Skipped",
                    "Reason": "Output file already exists",
                    "Path": destination_path
                })
                continue

            if not os.path.exists(destination):
                os.makedirs(destination)

            mt = hl.import_vcf(vcf, force_bgz=True, reference_genome='GRCh37', contig_recoding=contig_recoding,
                               skip_invalid_loci=True)
            mt = mt.filter_rows(mt.alleles[1] != "*")  # Filter star alleles
            print(f"Rows before filtering: {mt.count_rows()}")
            # mt = mt.filter_rows((hl.len(mt.filters) == 0)
            # mt.filters.contains(hl.literal("PASS"))) # - right now only for 255 VCFs that we wern't able to match mutations before

            #print(f"Rows after filtering PASS: {mt.count_rows()}") # uncomment if filtering with PASS is needed

            # Parse CSQ string and annotate rows
            # Modify the map lambda to check the string before parsing

            # First, make sure `vep` contains at least one element
            # Annotate rows with the `vep` field
            mt = mt.annotate_rows(
                vep=mt.info.CSQ.map(lambda csq: hl.struct(
                    IMPACT=safe_index(csq.split("|"), 0, "Unknown"),
                    SYMBOL=safe_index(csq.split("|"), 1, "Unknown"),
                    HGNC_ID=hl.if_else(safe_index(csq.split("|"), 2, "").matches(r'^-?\d+$'),
                                       parse_to_int32(safe_index(csq.split("|"), 2)), hl.missing(hl.tint32)),
                    MAX_AF=parse_to_float64(safe_index(csq.split("|"), 3, "0")),
                )),
                CHROM=mt.locus.contig,
                REF=mt.alleles[0],
                ALT=mt.alleles[1]
            )

            # Only keep rows where `vep` is not empty
            mt = mt.filter_rows(hl.len(mt.vep) > 0)
            # Flatten the first struct in the `vep` array directly into the row fields
            mt = mt.transmute_rows(
                IMPACT=mt.vep[0].IMPACT,
                SYMBOL=mt.vep[0].SYMBOL,
                HGNC_ID=mt.vep[0].HGNC_ID,
                MAX_AF=mt.vep[0].MAX_AF
            )
            #mt.info.CSQ.show()
            # After your processing is done and you have the MatrixTable `mt` ready
            # Define the TSV output path using the patient code and group
            df = mt.rows().flatten().to_pandas()
            # Process and filter necessary fields with `parse_csq_final`
            processed_data = parse_csq_final(df['info.CSQ'].tolist(), df['alleles'].tolist())
            filtered_df = pd.DataFrame(processed_data)
            # Add the columns to the DataFrame
            filtered_df['CHROM'] = df['CHROM']
            filtered_df['REF'] = df['alleles'].apply(lambda x: x[0])
            filtered_df['ALT'] = df['alleles'].apply(lambda x: x[1] if len(x) > 1 else None)
            filtered_df['QUAL'] = df['qual']
            for col in df.columns:
                if col.startswith('info.') and not col.endswith('CSQ'):
                    filtered_df[col.split('.')[1]] = df[col].apply(
                        lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

            if write:
                # Define the TSV output path and export
                tsv_output_path = os.path.join(destination, f"{patient_code}_mtoutput.tsv")
                filtered_df.to_csv(tsv_output_path, sep='\t', index=False)
                processed_vcfs.append({'vcf': vcf, 'mt_path': destination_path})
                log_entries.append({
                    "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "VCF": vcf,
                    "Status": "Processed",
                    "Reason": "Successfully processed and saved",
                    "Path": tsv_output_path
                })


    finally:
        hl.stop()  # Stop Hail context when done
        log_df = pd.DataFrame(log_entries)
        log_df.to_csv(log_file, sep='\t', index=False)

    return processed_vcfs, log_entries


# Example usage:
VEP_CONFIG_PATH = '/home/markus/gen-toolbox/src/config/vep_settings.json'
SOURCE_DIR = '/mnt/sdb/TSHC_data_5k'
base_destination_directory = '/mnt/sdb/markus-bsc-thesis-data'
phenotype_data = parse_phenotype_file('/mnt/sdb/phenotype-globals-tshc-7500.tsv') # file for one result per e-code
log_entries = vcfs_to_matrixtable(SOURCE_DIR, phenotype_data, base_destination_directory,
                                  write=True)  #TODO: this is broken, no logs are written down
print(log_entries)

## Data aggregation to create positive and negative group aggregated TSV table

In [5]:
# regex search from TSV files

import os
import pandas as pd
import re
from tqdm import tqdm

def valid_mutation_format(mutation):
    """ Check if the mutation string format is valid. Only proceed if mutation is a string. """
    if not isinstance(mutation, str):
        return False
    pattern = re.compile(r'NM_\d+\.\d+:c.*', re.IGNORECASE)
    return bool(pattern.match(mutation))

def extract_gene_and_mutation(mutation_info):
    try:
        gene_name, mutation_detail = mutation_info.split(':')
        mutation_pattern = re.compile(r'c\.\d+[+-]?\d*(?:[GATC]>[GATC]|del[ACTG]+|ins[ACTG]+|\d+_\d+del|\d+_\d+ins[ACTG]+)', re.IGNORECASE)
        matches = mutation_pattern.search(mutation_detail.strip())  # strip to remove any leading/trailing spaces
        if matches:
            return gene_name, matches.group()
        return gene_name, None
    except ValueError:
        return None, None

def aggregate_tsv_files(base_path, phenotype_dict, process_negative_group=True, process_positive_group=True, symbol=True, mutation=True):
    negative_group_path = os.path.join(base_path, 'negative-group')
    positive_group_path = os.path.join(base_path, 'positive-group-RV')
    neg_dfs = []
    pos_dfs = []
    val_dfs = []

    if process_negative_group:
        neg_files = os.listdir(negative_group_path)
        for file in tqdm(neg_files, desc="Processing Negative Group"):
            file_path = os.path.join(negative_group_path, file)
            df = pd.read_csv(file_path, sep='\t')
            e_code = file.split('_')[0]
            df['E_code'] = e_code
            neg_dfs.append(df)

    if process_positive_group:
        pos_files = os.listdir(positive_group_path)
        for file in tqdm(pos_files, desc="Processing Positive Group"):
            file_path = os.path.join(positive_group_path, file)
            df = pd.read_csv(file_path, sep='\t', low_memory=False)
            e_code = file.split('_')[0]
            df['E_code'] = e_code

            if e_code in phenotype_dict:
                _, _symbol, *mutations = phenotype_dict[e_code]
                mutations = [m for m in mutations if valid_mutation_format(m)]
                if mutations:  # Only proceed if there are valid mutations
                    subtracted_regex = None
                    if not mutation: # exclude mutations 
                        mutation_regex = '|'.join(re.escape(m.split(":")[0]) for m in mutations) 
                    else:
                        mutation_regex = '|'.join(re.escape(m) for m in mutations)
                        subtracted_mutation_regex = '|'.join(re.escape(m[:-2]) for m in mutations) # Pattern after removing last string from mutation
                        subtracted_regex = re.compile(subtracted_mutation_regex, re.IGNORECASE)
                        

                    regex = re.compile(mutation_regex, re.IGNORECASE)
                    
                    mask = df['MUTATIONS'].astype(str).apply(lambda x: subtracted_regex.search(x) is not None if regex.search(x) is None and mutation else regex.search(x) is not None)
                    df["nucleotide_expression"] = df["NM_STRING"] + ":" + df["MUTATIONS"] # Combine expression and mutation
                    df["secondary_matched"] = False # Field to track if mutation matcches the subtracted regex pattern
                    
                    if symbol and _symbol: # If symbol is true and e_code symbol is not null
                       df["matched"] = (df["nucleotide_expression"].str.contains(regex.pattern, na=False) & (df["SYMBOL"] == _symbol))
                       if subtracted_regex:
                           df["secondary_matched"] = (df["nucleotide_expression"].str.contains(subtracted_regex.pattern, na=False) & (df["SYMBOL"] == _symbol))
                    else:
                        df["matched"] = df["nucleotide_expression"].astype(str).apply(lambda x: regex.search(x) is not None)
                        if subtracted_regex:
                           df["secondary_matched"] = df["nucleotide_expression"].astype(str).apply(lambda x: regex.search(x) is not None)
                    
                    target_rows = df[(df["matched"] | ((df["matched"] == False) & (df["secondary_matched"]) & (df["IMPACT"] == "HIGH")))] # rows that match the expression
                    if not target_rows.empty:
                        df.loc[mask, 'E_code'] = e_code
                        pos_dfs.append(target_rows)
                    else:
                        # If no matching mutation is found, consider it for validation group
                        val_dfs.append(df)

    # Aggregate and save DataFrames as done previously
    # Combine all DataFrames for the negative group and save
    if neg_dfs:
        negative_agg_df = pd.concat(neg_dfs, ignore_index=True)
        #negative_agg_df.drop(["nucleotide_expression", "matched"], inplace=True, axis=1)
        negative_agg_df.to_csv(os.path.join(base_path, 'negative_group_aggregated.tsv'), sep='\t', index=False)
        print("Negative group data saved successfully.")

    # Combine all DataFrames for the positive group and save if there are any
    if pos_dfs:
        positive_agg_df = pd.concat(pos_dfs, ignore_index=True)
        positive_agg_df.drop(["nucleotide_expression", "matched", "secondary_matched"], inplace=True, axis=1)
        positive_agg_df.to_csv(os.path.join(base_path, 'positive_group_aggregated.tsv'), sep='\t', index=False)
        print("Positive group data saved successfully.")
        
    if val_dfs:
        print("Aga siia")
        validation_agg_df = pd.concat(val_dfs, ignore_index=True)
        validation_agg_df.drop(["nucleotide_expression", "matched", "secondary_matched"], inplace=True, axis=1)
        validation_agg_df.to_csv(os.path.join(base_path, 'validation_group_aggregated.tsv'), sep='\t', index=False)
        print("Validation group data saved successfully.")
        
    else:
        print("No data for positive group or all data were non-matching.")


def regex_search(symbol=True, mutation=True, additional_nucleotide_expressions=True):
    # Assuming phenotype_data is loaded properly
    phenotype_data = parse_phenotype_file_HGSV('/mnt/sdb/markus-bsc-thesis-data/cleaned-HGSV-phenotype-globals-tshc-7500.tsv', additional_nucleotide_expressions)
    aggregate_tsv_files('/mnt/sdb/markus-bsc-thesis-data/', phenotype_data, process_negative_group=True, process_positive_group=True, symbol=symbol, mutation=mutation)

regex_search(symbol=True, mutation=True, additional_nucleotide_expressions=True)

Processing Positive Group: 100%|██████████| 355/355 [00:27<00:00, 13.02it/s]


Negative group data saved successfully.
Positive group data saved successfully.
Aga siia
Validation group data saved successfully.
